## Σε αυτό το σημείο της εργασίες πρέπει να συγκρίνουμε τα 3 pipelines που χρησιμοποιήσαμε στο ερώτημα 1Β μεταξύ τους

### Βήματα 
- Διαβάζουμε τα 3 ανακατσκευασμένα κείμενα
- Κάνουμε σύγκριση

In [8]:
original_text=""
pipeline1=""
pipeline2=""
pipeline3=""

In [9]:
try:
    with open('../text_files/text1.txt', 'r', encoding='utf-8') as file:
        original_text = file.read()
    print("File content successfully loaded into original_text.")
except FileNotFoundError:
    print("Error: The file 'text1.txt' was not found. Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into original_text.


In [10]:
try:
    with open("../text_files/first_pipeline_text1", 'r', encoding='utf-8') as file:
        pipeline1 = file.read()
    print("File content successfully loaded into pipeline1.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline1.


In [11]:
try:
    with open("../text_files/second_pipeline_text1", 'r', encoding='utf-8') as file:
        pipeline2 = file.read()
    print("File content successfully loaded into pipeline2.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline2.


In [12]:
try:
    with open("../text_files/third_pipeline_text1", 'r', encoding='utf-8') as file:
        pipeline3 = file.read()
    print("File content successfully loaded into pipeline3.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline3.


## Αρχικά θα συγκρίνουμε το πόσο διαφέρει η κάθε ανακατασκευή από το πρωτότυπο κείμενο με μέθοδο cosine similarity

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = model.encode([original_text, pipeline1, pipeline2, pipeline3])

similarities = {
    "Pipeline 1": util.cos_sim(embeddings[0], embeddings[1]).item(),
    "Pipeline 2": util.cos_sim(embeddings[0], embeddings[2]).item(),
    "Pipeline 3": util.cos_sim(embeddings[0], embeddings[3]).item()
}


for name, score in similarities.items():
    print(f"{name}: Cosine Similarity = {score:.4f}")

c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\alexi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alexi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

Pipeline 1: Cosine Similarity = 0.9977
Pipeline 2: Cosine Similarity = 0.9927
Pipeline 3: Cosine Similarity = 0.8468
